<a href="https://colab.research.google.com/github/jjpal/dk122022_data_dive/blob/main/SNAP__SVI__Map_Meal_gap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

### SUPPLEMENTAL NUTRITION ASSISTANCE PROGRAM (SNAP)

Column Headers

Substate, PersonsPublic, PersonsNonPublic, PersonsTotal,HouseholdsPublicAssistance, HouseholdsNonPublicAssistance, HouseholdsTotal,Issuance, month, year, countyFIPS, countyNAME, stateFIPS, stateNAME

In [2]:
# Processed Merged SNAP data
snap_merge_url = 'https://raw.githubusercontent.com/michaelbbryan/tools-and-kaggles/main/DataDive2022/snapmergecounty.csv'


In [3]:
# Load SNAP dataset
snap_merge_df = pd.read_csv(snap_merge_url)

In [4]:
# Only select year 2020 to match up with the other datasets
snap_county_2020_df = snap_merge_df[snap_merge_df['year'] == 2020]

In [5]:
# groupby on multiple columns and calculate a sum over each combination group.
snap_county_2020_dfnew = snap_county_2020_df.groupby(['countyFIPS','year']).sum().reset_index()

In [6]:
# display sample of dataset
snap_county_2020_df.sample(3)

,Substate,PersonsPublic,PersonsNonPublic,PersonsTotal,HouseholdsPublicAssistance,HouseholdsNonPublicAssistance,HouseholdsTotal,Issuance,month,year,countyFIPS,countyNAME,stateFIPS,stateNAME
179276,3702702 NC EBT CALDWELL,184.0,12372.0,12556.0,78.0,6251.0,6329.0,1405573.0,JUL,2020,37027,Caldwell County,37,NORTH CAROLIN
88081,3905301 OH EBT GALLIA CO DHS,1116.0,5255.0,6371.0,863.0,2360.0,3223.0,762925.0,JAN,2020,39053,Gallia County,39,OHIO
180377,5515301 WI EBT SOKAOGON TRIBAL AGENCY,11.0,163.0,174.0,11.0,72.0,83.0,20073.0,JUL,2020,55153,0,0,0


In [7]:
snap_county_2020_df.month.value_counts()

JAN    2626
JUL    2626
Name: month, dtype: int64

### Social Vulnerability Indicators (selected columnss)


In [8]:
# social vulnerability indicators for 2020
svi2020_url = 'https://raw.githubusercontent.com/jjpal/dk122022_data_dive/main/SVI2020_US_COUNTY.csv'

# load svi dataset with selected columns
svi_2020_df = pd.read_csv(svi2020_url, usecols = ['STATE','ST_ABBR','COUNTY','FIPS','RPL_THEME1', 'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'RPL_THEMES'])

# display sample of dataset
svi_2020_df.sample(3)

,STATE,ST_ABBR,COUNTY,FIPS,RPL_THEME1,RPL_THEME2,RPL_THEME3,RPL_THEME4,RPL_THEMES
1762,Nevada,NV,Washoe,32031,0.6359,0.3873,0.7689,0.7651,0.6836
2346,South Carolina,SC,Laurens,45059,0.9615,0.7613,0.7110,0.8447,0.9144
444,Georgia,GA,Floyd,13115,0.9306,0.9179,0.6785,0.9675,0.9679


In [9]:
# merge previously merged SNAP data with svi2020 indicators
SNAP_SVI = pd.merge(snap_county_2020_dfnew, svi_2020_df,left_on='countyFIPS',right_on='FIPS',how='inner')

### Map Meal Gap Data 

In [10]:
# map_meal_gap_data - with County sheet name
map_meal_gap_2020_df = pd.read_excel(io='https://github.com/jjpal/dk122022_data_dive/blob/main/MMG2022_2020-2019Data_ToShare.xlsx?raw=true', sheet_name ='County')

In [11]:
# separate County and State into different columns
map_meal_gap_2020_df[['County', 'States']] = map_meal_gap_2020_df['County, State'].str.split(pat = ',', expand = True)

map_meal_gap_2020_df = map_meal_gap_2020_df[map_meal_gap_2020_df['Year'] == 2020]
map_meal_gap_2020_df.drop('County, State', axis=1, inplace=True)

In [12]:
# change the datatype to datetime
map_meal_gap_2020_df['Year'] = pd.to_datetime(map_meal_gap_2020_df['Year'], format='%Y')                            

In [13]:
# change the datatype
map_meal_gap_2020_df['FIPS'] = map_meal_gap_2020_df['FIPS'].astype(int)

In [14]:
# combine previously merged snap_svi df with map meal datase
map_svi_snap = pd.merge(SNAP_SVI, map_meal_gap_2020_df,left_on='FIPS',right_on='FIPS',how='inner')

In [15]:
print(map_svi_snap.shape)
print(map_svi_snap.columns)

(2507, 42)
Index(['countyFIPS', 'year', 'PersonsPublic', 'PersonsNonPublic',
       'PersonsTotal', 'HouseholdsPublicAssistance',
       'HouseholdsNonPublicAssistance', 'HouseholdsTotal', 'Issuance',
       'stateFIPS', 'STATE', 'ST_ABBR', 'COUNTY', 'FIPS', 'RPL_THEME1',
       'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'RPL_THEMES', 'State', 'Year',
       'Overall Food Insecurity Rate (1 Year)',
       '# of Food Insecure Persons Overall (1 Year)',
       'Food Insecurity Rate among Black Persons (all ethnicities)',
       'Food Insecurity Rate among Hispanic Persons (any race)',
       'Food Insecurity Rate among White, non-Hispanic Persons ',
       'Low Threshold in state', 'Low Threshold Type',
       'High Threshold in state', 'High Threshold Type',
       '% FI ≤ Low Threshold', '% FI Btwn Thresholds', '% FI > High Threshold',
       'Child Food Insecurity Rate (1 Year)',
       '# of Food Insecure Children (1 Year)',
       '% food insecure children in HH w/ HH incomes below 1

In [16]:
# FIPS CODE B11001 from census households per county
census_table_url = 'https://raw.githubusercontent.com/michaelbbryan/tools-and-kaggles/main/DataDive2022/juanita.csv'

# load svi dataset with selected columns
census_FIPS_df = pd.read_csv(census_table_url, usecols = ['countyFIPS', 'HHCount'])

# display dataframe
census_FIPS_df.sample(3) 

,countyFIPS,HHCount
1984,37187,5237.0
1298,26133,9332.0
1198,24011,12160.0


In [17]:
# merge FIPS column to existing merged data and assign to a new df
census_map_svi_snap = pd.merge(census_FIPS_df, map_svi_snap,left_on='countyFIPS',right_on='countyFIPS',how='inner')

# display dataframe
census_map_svi_snap.sample(3)

,countyFIPS,HHCount,year,PersonsPublic,PersonsNonPublic,PersonsTotal,HouseholdsPublicAssistance,HouseholdsNonPublicAssistance,HouseholdsTotal,Issuance,...,% FI > High Threshold,Child Food Insecurity Rate (1 Year),# of Food Insecure Children (1 Year),% food insecure children in HH w/ HH incomes below 185 FPL,% food insecure children in HH w/ HH incomes above 185 FPL,Cost Per Meal (1 Year),Weighted weekly $ needed by FI,Weighted Annual Food Budget Shortfall,County,States
2117,48245,94012.0,2020,513.0,83207.0,83720.0,197.0,39529.0,39726.0,12509785.0,...,0.354,0.318,19360.0,0.66,0.34,2.81,14.931277,20866000.0,Jefferson County,Texas
2199,48409,23422.0,2020,168.0,23775.0,23943.0,54.0,10142.0,10196.0,3467105.0,...,0.190,0.254,4570.0,0.75,0.25,2.88,15.270624,5021000.0,San Patricio County,Texas
178,6043,7846.0,2020,744.0,3736.0,4480.0,462.0,2197.0,2659.0,673925.0,...,0.228,0.228,650.0,0.67,0.33,4.00,21.230566,1662000.0,Mariposa County,California


In [18]:
# calculation SNAP HouseholdsTotal/#census_household
census_map_svi_snap['HH_ratio'] = census_map_svi_snap['HouseholdsTotal'] / census_map_svi_snap['HHCount']

In [19]:
# Columns of 4 table merge of selected columns
census_map_svi_snap.columns

Index(['countyFIPS', 'HHCount', 'year', 'PersonsPublic', 'PersonsNonPublic',
       'PersonsTotal', 'HouseholdsPublicAssistance',
       'HouseholdsNonPublicAssistance', 'HouseholdsTotal', 'Issuance',
       'stateFIPS', 'STATE', 'ST_ABBR', 'COUNTY', 'FIPS', 'RPL_THEME1',
       'RPL_THEME2', 'RPL_THEME3', 'RPL_THEME4', 'RPL_THEMES', 'State', 'Year',
       'Overall Food Insecurity Rate (1 Year)',
       '# of Food Insecure Persons Overall (1 Year)',
       'Food Insecurity Rate among Black Persons (all ethnicities)',
       'Food Insecurity Rate among Hispanic Persons (any race)',
       'Food Insecurity Rate among White, non-Hispanic Persons ',
       'Low Threshold in state', 'Low Threshold Type',
       'High Threshold in state', 'High Threshold Type',
       '% FI ≤ Low Threshold', '% FI Btwn Thresholds', '% FI > High Threshold',
       'Child Food Insecurity Rate (1 Year)',
       '# of Food Insecure Children (1 Year)',
       '% food insecure children in HH w/ HH incomes below 1

In [20]:
census_map_svi_snap.head(3)

,countyFIPS,HHCount,year,PersonsPublic,PersonsNonPublic,PersonsTotal,HouseholdsPublicAssistance,HouseholdsNonPublicAssistance,HouseholdsTotal,Issuance,...,Child Food Insecurity Rate (1 Year),# of Food Insecure Children (1 Year),% food insecure children in HH w/ HH incomes below 185 FPL,% food insecure children in HH w/ HH incomes above 185 FPL,Cost Per Meal (1 Year),Weighted weekly $ needed by FI,Weighted Annual Food Budget Shortfall,County,States,HH_ratio
0,1001,21559.0,2020,1977.0,13108.0,15085.0,1064.0,5764.0,6828.0,2219777.0,...,0.181,2380.0,0.71,0.29,3.22,17.092792,4184000.0,Autauga County,Alabama,0.316712
1,1003,84047.0,2020,3723.0,37358.0,41081.0,2229.0,16252.0,18481.0,6120282.0,...,0.120,5650.0,0.82,0.18,3.72,19.723135,16195000.0,Baldwin County,Alabama,0.219889
2,1005,9322.0,2020,1607.0,9683.0,11290.0,960.0,4280.0,5240.0,1660768.0,...,0.334,1740.0,0.82,0.18,3.19,16.956475,2505000.0,Barbour County,Alabama,0.562111
